In [2]:
import os
import numpy as np
from PIL import Image

# Loading Data

In [3]:
images = []
ages = []
genders = []
races = []
for file in os.listdir('/kaggle/input/utkface-new/UTKFace'):
    split = file.split('_')
    if len(split)==4:
        if int(split[0]) in (103,111):
            continue
        ages.append(int(split[0]))
        genders.append(int(split[1]))
        races.append(int(split[2]))
        img = Image.open(os.path.join('/kaggle/input/utkface-new/UTKFace',file))
        img = img.resize((128,128), Image.LANCZOS)
        images.append(np.array(img))

images = np.array(images)
ages = np.array(ages)
genders = np.array(genders)
races = np.array(races)

# Gender Model

In [5]:
from sklearn.model_selection import train_test_split

x_train_age, x_test_age, y_train_age, y_test_age = train_test_split(images,
                                                                    ages, test_size=0.2, 
                                                                    stratify=ages)

In [6]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Activation, Dropout, Flatten, Dense
from tensorflow.keras import optimizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping

model_age = Sequential()
model_age.add(Conv2D(32, (3,3), activation='relu', input_shape=(128, 128, 3)))
model_age.add(MaxPooling2D((2,2)))
model_age.add(Conv2D(64, (3,3), activation='relu'))
model_age.add(MaxPooling2D((2,2)))
model_age.add(Conv2D(128, (3,3), activation='relu'))
model_age.add(MaxPooling2D((2,2)))
model_age.add(Flatten())
model_age.add(Dense(64, activation='relu'))
model_age.add(Dropout(0.2))
model_age.add(Dense(1, activation='linear'))  

model_age.compile(
    loss='mse',
    optimizer=optimizers.Adam(learning_rate=0.001),
    metrics=['mae']
)

train_datagen_age = ImageDataGenerator(
      rescale=1./255., width_shift_range = 0.1, 
      height_shift_range = 0.1, horizontal_flip = True)

test_datagen_age = ImageDataGenerator(rescale=1./255)

train_age = train_datagen_age.flow(x_train_age, y_train_age, batch_size=32)

test_age = test_datagen_age.flow(
        x_test_age, y_test_age,
        batch_size=32,
        shuffle = False)

early_stop = EarlyStopping(
    monitor='val_loss',     # monitor validation loss
    patience=5,             # stop after 5 epochs of no improvement
    restore_best_weights=True # restore weights from the best epoch
)

history_age = model_age.fit(
    train_age,
    epochs=50,
    shuffle=True,
    validation_data=test_age,
    callbacks=[early_stop]
)


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/50
593/593 ━━━━━━━━━━━━━━━━━━━━ 66s 106ms/step - loss: 462.4433 - mae: 16.4563 - val_loss: 293.9743 - val_mae: 12.7782
Epoch 2/50
593/593 ━━━━━━━━━━━━━━━━━━━━ 60s 101ms/step - loss: 290.9258 - mae: 13.1675 - val_loss: 191.9577 - val_mae: 10.2547
Epoch 3/50
593/593 ━━━━━━━━━━━━━━━━━━━━ 60s 101ms/step - loss: 225.1427 - mae: 11.3895 - val_loss: 186.7205 - val_mae: 10.6904
Epoch 4/50
593/593 ━━━━━━━━━━━━━━━━━━━━ 60s 102ms/step - loss: 202.3829 - mae: 10.7666 - val_loss: 140.2190 - val_mae: 8.8261
Epoch 5/50
593/593 ━━━━━━━━━━━━━━━━━━━━ 61s 102ms/step - loss: 183.8994 - mae: 10.2419 - val_loss: 128.5907 - val_mae: 8.3732
Epoch 6/50
593/593 ━━━━━━━━━━━━━━━━━━━━ 61s 102ms/step - loss: 172.1923 - mae: 9.9252 - val_loss: 140.7571 - val_mae: 9.1549
Epoch 7/50
593/593 ━━━━━━━━━━━━━━━━━━━━ 61s 103ms/step - loss: 162.6128 - mae: 9.5810 - val_loss: 118.1170 - val_mae: 8.2945
Epoch 8/50
593/593 ━━━━━━━━━━━━━━━━━━━━ 60s 102ms/step - loss: 148.3581 - mae: 9.1576 - val_loss: 120.0150 - val_mae:

In [8]:
model_age.save('model_age.h5')

In [48]:
from sklearn.metrics import mean_absolute_error, mean_squared_error
test_age = test_datagen_age.flow(
        x_test_age, y_test_age,
        batch_size=32,
        shuffle =False)
y_pred = loaded_model.predict(test_age, verbose=0).squeeze()
y_true = y_test_age

# Compute metrics
mae = mean_absolute_error(y_true, y_pred)
rmse = np.sqrt(mean_squared_error(y_true, y_pred))

print(f"MAE: {mae}.")
print(f"RMSE: {rmse}.")

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


MAE: 6.7037061066900065.
RMSE: 9.239504403118906.


# Gender Model

In [6]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Activation, Dropout, Flatten, Dense
from tensorflow.keras import optimizers, losses
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split

x_train_gender, x_test_gender, y_train_gender, y_test_gender = train_test_split(images,
                                                                    genders, test_size=0.2, 
                                                                    stratify=genders)


model_gender = Sequential()
model_gender.add(Conv2D(32, (3,3), activation='relu', input_shape=(128, 128, 3)))
model_gender.add(MaxPooling2D((2,2)))
model_gender.add(Conv2D(64, (3,3), activation='relu'))
model_gender.add(MaxPooling2D((2,2)))
model_gender.add(Conv2D(128, (3,3), activation='relu'))
model_gender.add(MaxPooling2D((2,2)))
model_gender.add(Flatten())
model_gender.add(Dense(64, activation='relu'))
model_gender.add(Dropout(0.2))
model_gender.add(Dense(1, activation='sigmoid'))

model_gender.compile(
    loss=losses.BinaryCrossentropy(),
    optimizer=optimizers.Adam(learning_rate=0.001),
    metrics=['accuracy']
)

train_datagen_gender = ImageDataGenerator(
      rescale=1./255., width_shift_range = 0.1, 
      height_shift_range = 0.1, horizontal_flip = True)

test_datagen_gender = ImageDataGenerator(rescale=1./255)

train_gender = train_datagen_gender.flow(x_train_gender, y_train_gender, batch_size=32)

test_gender = test_datagen_gender.flow(
        x_test_gender, y_test_gender,
        batch_size=32,
        shuffle = False)

early_stop = EarlyStopping(
    monitor='val_loss',     # monitor validation loss
    patience=5,             # stop after 5 epochs of no improvement
    restore_best_weights=True # restore weights from the best epoch
)

history_gender = model_gender.fit(
    train_gender,
    epochs=50,
    shuffle=True,
    validation_data=test_gender,
    callbacks=[early_stop]
)


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/50


I0000 00:00:1756475252.631005     100 service.cc:148] XLA service 0x7cb334005e80 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1756475252.631746     100 service.cc:156]   StreamExecutor device (0): Tesla P100-PCIE-16GB, Compute Capability 6.0
I0000 00:00:1756475252.962289     100 cuda_dnn.cc:529] Loaded cuDNN version 90300


  3/593 ━━━━━━━━━━━━━━━━━━━━ 35s 60ms/step - accuracy: 0.4792 - loss: 0.6934

I0000 00:00:1756475255.822189     100 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


593/593 ━━━━━━━━━━━━━━━━━━━━ 69s 108ms/step - accuracy: 0.6251 - loss: 0.6369 - val_accuracy: 0.8000 - val_loss: 0.4459
Epoch 2/50
593/593 ━━━━━━━━━━━━━━━━━━━━ 61s 103ms/step - accuracy: 0.7627 - loss: 0.4980 - val_accuracy: 0.8264 - val_loss: 0.3840
Epoch 3/50
593/593 ━━━━━━━━━━━━━━━━━━━━ 61s 102ms/step - accuracy: 0.7832 - loss: 0.4541 - val_accuracy: 0.8408 - val_loss: 0.3629
Epoch 4/50
593/593 ━━━━━━━━━━━━━━━━━━━━ 60s 101ms/step - accuracy: 0.7900 - loss: 0.4358 - val_accuracy: 0.8386 - val_loss: 0.3641
Epoch 5/50
593/593 ━━━━━━━━━━━━━━━━━━━━ 60s 100ms/step - accuracy: 0.8106 - loss: 0.4114 - val_accuracy: 0.8536 - val_loss: 0.3213
Epoch 6/50
593/593 ━━━━━━━━━━━━━━━━━━━━ 60s 101ms/step - accuracy: 0.8141 - loss: 0.3940 - val_accuracy: 0.8604 - val_loss: 0.3168
Epoch 7/50
593/593 ━━━━━━━━━━━━━━━━━━━━ 60s 102ms/step - accuracy: 0.8208 - loss: 0.3880 - val_accuracy: 0.8559 - val_loss: 0.3128
Epoch 8/50
593/593 ━━━━━━━━━━━━━━━━━━━━ 60s 101ms/step - accuracy: 0.8339 - loss: 0.3676 - val

In [8]:
model_gender.save('model_gender.h5')

In [10]:
from sklearn.metrics import classification_report, accuracy_score

y_pred = model_gender.predict(test_gender, verbose=0).squeeze()
y_pred = (y_pred > 0.5).astype("int32") 
y_true = y_test_gender

# Full classification report
print("\nClassification Report:")
print(classification_report(y_true, y_pred, digits=4))



Classification Report:
              precision    recall  f1-score   support

           0     0.9206    0.9124    0.9165      2478
           1     0.9050    0.9138    0.9094      2263

    accuracy                         0.9131      4741
   macro avg     0.9128    0.9131    0.9130      4741
weighted avg     0.9132    0.9131    0.9131      4741

